# TensorFlow Script Mode

O modo de script é um formato de script de treinamento para o TensorFlow que permite executar qualquer script de treinamento do TensorFlow no SageMaker com modificações mínimas. O [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) lida com a transferência de seu script para uma instância de treinamento. Na instância de treinamento, o suporte nativo do TensorFlow do SageMaker configura variáveis de ambiente relacionadas ao treinamento e executa seu script.

O modo de script oferece suporte ao treinamento com um script Python, um módulo Python ou um script shell. Neste exemplo, usamos um script Python para treinar um modelo de classificação no [conjunto de dados MNIST](http://yann.lecun.com/exdb/mnist/). Podemos usar os scripts em TensorFlow 1.x e TensorFlow 2.0 com, além disso, este iremos demonstrar como realizar inferências em tempo real com o [Container SageMaker TensorFlow Serving](https://github.com/aws/sagemaker-tensorflow-serving-container).


## Configurando o ambiente

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

## Dados de treinamento

O dataset MNIST já está em um bucket S3 ``sagemaker-sample-data-<REGION>`` com o prefixo ``tensorflow/mnist``:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
training_data_uri = f's3://sagemaker-sample-data-{region}/tensorflow/mnist'

# Construindo o script


O script de treinamento deste tutorial foi adaptado do [exemplo CNN MNIST oficial do TensorFlow](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). Nós o modificamos para lidar com o parâmetro `` model_dir`` passado pelo SageMaker. Este é um caminho S3 que pode ser usado para compartilhamento de dados durante o treinamento distribuído e checkpoints e/ou persistência do modelo. Também adicionamos uma função de análise de parâmetros para lidar com o processamento de variáveis relacionadas ao treinamento.

No final do job de treinamento, adicionamos uma etapa para exportar o modelo treinado para o caminho armazenado na variável de ambiente `` SM_MODEL_DIR``, que sempre aponta para ``/opt/ml/model``. Isso é crítico porque o SageMaker carrega todos os artefatos do modelo nesta pasta para o S3 no final do treinamento.

Aqui está o script completo:

In [ ]:
# TensorFlow 2.1 script
!pygmentize 'mnist-2.py'

## Criando um treinamento com o `TensorFlow` Estimator

O Estimator `sagemaker.tensorflow.TensorFlow` lida com a localização do contêiner do script mode, enviando seu script para um local S3 e criando um trabalho de treinamento no SageMaker. Vamos ver alguns parâmetros importantes:

* `py_version` é definido como` 'py3'` para indicar que estamos usando o modo de script, já que o modo legado suporta apenas Python 2. Embora Python 2 seja obsoleto em breve, você pode usar o modo de script com Python 2 definindo `py_version` para `'py2'` e` script_mode` para `True`.

* `distributions` é usado para configurar a configuração de treinamento distribuído. É necessário apenas se você estiver fazendo treinamento distribuído em um cluster de instâncias ou em várias GPUs. Aqui, estamos usando servidores de parâmetros como esquema de treinamento distribuído. Os jobs de treinamento do SageMaker são executados em clusters homogêneos. Para tornar o servidor de parâmetros mais eficiente na configuração do SageMaker, executamos um servidor de parâmetros em cada instância do cluster, portanto, não há necessidade de especificar o número de servidores de parâmetros a serem iniciados. O modo de script também oferece suporte a treinamento distribuído com [Horovod](https://github.com/horovod/horovod). Você pode encontrar a documentação completa sobre como configurar `distribuições` [aqui](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training).

In [ ]:
mnist_estimator2 = TensorFlow(entry_point='mnist-2.py',
                             role=role,
                             train_instance_count=2,
                             train_instance_type='ml.p3.2xlarge',
                             framework_version='2.1.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

## Iniciando o treinamento (``fit``)

Para iniciar um job de treinamento, chamamos `estimator.fit (training_data_uri)`.

Um local S3 é usado como entrada do `fit`. Cria-se um canal padrão chamado` 'training'`, que aponta para este local S3. No script de treinamento, podemos acessar os dados de treinamento do local armazenado em `SM_CHANNEL_TRAINING`. `fit` aceita alguns outros tipos de entrada também. Consulte a documentação da API [aqui](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) para obter detalhes.

Quando o treinamento começa, o contêiner TensorFlow executa o script mnist2.py, passando `hiperparâmetros` e` model_dir` do estimador como argumentos de script. Como não definimos nenhum deles neste exemplo, nenhum hiperparâmetro é passado, e `model_dir` é padronizado como` s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, então a execução do script é a seguinte:

```bash
python mnist.py --model_dir s3: // <DEFAULT_BUCKET> / <TRAINING_JOB_NAME>
```

Quando o treinamento for concluído, o job de treinamento fará o upload do modelo

In [ ]:
mnist_estimator2.fit(training_data_uri)

## Efetuando o deploy do modelo

In [ ]:
predictor2 = mnist_estimator2.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

# Chamando o endpoint

In [ ]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

In [ ]:
predictions2 = predictor2.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions2['predictions'][i]
    label = train_labels[i]
    print('predição é {}, rótulo é {}, match: {}'.format(prediction, label, prediction == label))

## Deletando o endpoint

In [ ]:
#sagemaker.Session().delete_endpoint(predictor2.endpoint)